In [1]:
pip install nyoka

   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------------- -------------------- 1.8/3.8 MB 10.0 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 9.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from nyoka import skl_to_pmml
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)
cols = load_diabetes()['feature_names']

scaler = MinMaxScaler()
pipe = Pipeline([  
            ('Scaling', MinMaxScaler()),
            ('Linear', LinearRegression())
        ])
# Обучение пайплайна, включающего линейную модель и нормализацию признаков
pipe.fit(X, y)
# Сохраним пайплайн в формате pmml в файл pipeline.pmml
skl_to_pmml(pipeline=pipe, col_names=cols, pmml_f_name="pipeline.pmml")

In [4]:
pip install onnxruntime


   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 7.1 MB/s eta 0:00:02
   ------------ --------------------------- 3.4/11.1 MB 10.1 MB/s eta 0:00:01
   --------------------- ------------------ 6.0/11.1 MB 10.8 MB/s eta 0:00:01
   ------------------------ --------------- 6.8/11.1 MB 8.9 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.1 MB 7.9 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.1 MB 6.7 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.1 MB 6.4 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.1 MB 5.8 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.1 MB 5.3 MB/s eta 0:00:01
   ------------------------------------ --- 10.2/11.1 MB 5.1 MB/s eta 0:00:01
   ---------------------------------------  11.0/11.1 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 4.7 MB/s eta 0:00:00


In [5]:
pip install skl2onnx

   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/14.5 MB 8.3 MB/s eta 0:00:02
   ----------- ---------------------------- 4.2/14.5 MB 10.9 MB/s eta 0:00:01
   ---------------- ----------------------- 6.0/14.5 MB 10.8 MB/s eta 0:00:01
   ------------------ --------------------- 6.8/14.5 MB 8.6 MB/s eta 0:00:01
   -------------------- ------------------- 7.3/14.5 MB 7.5 MB/s eta 0:00:01
   ---------------------- ----------------- 8.1/14.5 MB 6.5 MB/s eta 0:00:01
   ----------------------- ---------------- 8.7/14.5 MB 6.0 MB/s eta 0:00:01
   ------------------------- -------------- 9.4/14.5 MB 5.6 MB/s eta 0:00:01
   --------------------------- ------------ 10.0/14.5 MB 5.4 MB/s eta 0:00:01
   ----------------------------- ---------- 10.7/14.5 MB 5.0 MB/s eta 0:00:01
   ------------------------------- -------- 11.3/14.5 MB 4.9 MB/s eta 0:00:01
   -------------------------------- ------- 11.8/14.5 MB 4.7 MB/s eta 0:00:01


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.


In [6]:
import onnxruntime as rt 
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType


# загружаем данные
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7)
print(X_train.shape, X_test.shape)

# обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)

# делаем инференс моделью на тесте
test_pred = model.predict(X_test)
print('sklearn model predict:\n', test_pred)

# конвертируем модель в ONNX-формат
initial_type = [('float_input', FloatTensorType([None, 13]))]
model_onnx = convert_sklearn(model, initial_types=initial_type)

# сохраняем модель в файл
with open("model.onnx", "wb") as f:
	f.write(model_onnx.SerializeToString())
 	 
# Делаем инференс на тесте через ONNX-runtime
sess = rt.InferenceSession("model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
test_pred_onnx = sess.run([label_name],
                	{input_name:  X_test.astype(np.float32)})[0].reshape(-1)
print('onnx model predict:\n',test_pred_onnx)

ImportError: 
`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[: